<a href="https://colab.research.google.com/github/shorokmohamed/shorok-mohamed-ahmed/blob/main/Graph_Based_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
pip install git+https://github.com/boudinfl/pke.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/boudinfl/pke.git to /tmp/pip-req-build-lbucwm8h
  Running command git clone -q https://github.com/boudinfl/pke.git /tmp/pip-req-build-lbucwm8h
     |████████████████████████████████| 235 kB 4.9 MB/s 
  Created wheel for pke: filename=pke-2.0.0-py3-none-any.whl size=6160276 sha256=f1e518fc032e674cbfd3b9f1834775a0be94069fe4c14cefeb9fb65dfd580246
  Stored in directory: /tmp/pip-ephem-wheel-cache-0oabiv94/wheels/fa/b3/09/612ee93bf3ee4164bcd5783e742942cdfc892a86039d3e0a33
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1310 sha256=9c2d56835cb7dd1f88c73b30a4ba8c89901cb82d477ef0f86908b8dbbde0ff57
  Stored in directory: /root/.cache/pip/wheels/46/ef/c3/157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e
Successfully built pke sklearn


In [4]:
import pke

In [5]:
data= """Coronaviruses are a group of related RNA viruses that cause diseases in mammals and birds.In humans and birds, they cause respiratory tract infections that can
range from mild to lethal. Mild illnesses in humans include some cases of the common cold (which is also caused by other viruses, predominantly rhinoviruses), while
more lethal varieties can cause SARS, MERS and COVID-19, which is causing the ongoing pandemic. In cows and pigs they cause diarrhea, while in mice they cause 
hepatitis and encephalomyelitis.
Coronaviruses constitute the subfamily Orthocoronavirinae, in the family Coronaviridae, order Nidovirales and realm Riboviria. They are enveloped viruses with a
positive-sense single-stranded RNA genome and a nucleocapsid of helical symmetry. The genome size of coronaviruses ranges from approximately 26 to 32 kilobases,
one of the largest among RNA viruses. They have characteristic club-shaped spikes that project from their surface, which in electron micrographs create an image
reminiscent of the stellar corona, from which their name derives.
The name "coronavirus" is derived from Latin corona, meaning "crown" or "wreath", itself a borrowing from Greek κορώνη korṓnē, "garland, wreath". The name was coined
by June Almeida and David Tyrrell who first observed and studied human coronaviruses. The word was first used in print in 1968 by an informal group of virologists in
the journal Nature to designate the new family of viruses. The name refers to the characteristic appearance of virions (the infective form of the virus) by electron
microscopy, which have a fringe of large, bulbous surface projections creating an image reminiscent of the solar corona or halo. This morphology is created by the 
viral spike peplomers, which are proteins on the surface of the virus.
The scientific name Coronavirus was accepted as a genus name by the International Committee for the Nomenclature of Viruses (later renamed International Committee on
Taxonomy of Viruses) in 1971. As the number of new species increased, the genus was split into four genera, namely Alphacoronavirus, Betacoronavirus, Deltacoronavirus,
and Gammacoronavirus in 2009. The common name coronavirus is used to refer to any member of the subfamily Orthocoronavirinae. As of 2020, 45 species are officially
recognised.""".replace('\n' , " ")

In [6]:
graph=[]

# TopicRank Model

In [7]:
extractor = pke.unsupervised.TopicRank()
extractor.load_document(input=data , language='en')

#Extract Sentences List
for i, sentence in enumerate(extractor.sentences):
    
    print("sentence {}:".format(i))
    print(" - words: {} ...".format(' '.join(sentence.words[:5])))
    print(" - stems: {} ...".format(' '.join(sentence.stems[:5])))     # back to  word to its original
    print(" - PoS: {} ...".format(' '.join(sentence.pos[:5])))         #  show the word is noun, verb, adjective 


## Extract Candidate from the **document**

extractor.candidate_selection()
for i, candidate in enumerate(extractor.candidates):
     
    print("candidate {}: {} (stemmed form)".format(i, candidate))
    print(" - surface forms:", [ " ".join(u) for u in extractor.candidates[candidate].surface_forms])
    print(" - offsets:", extractor.candidates[candidate].offsets)
    print(" - sentence_ids:", extractor.candidates[candidate].sentence_ids)
    print(" - pos_patterns:", extractor.candidates[candidate].pos_patterns)



## Grouping keyphrase candidates into topics 
extractor.candidate_weighting()

## **Topic of the document**
for i, topic in enumerate(extractor.topics):
    
    # print out the topic id and the candidates it groups together
    print("topic {}: {} ".format(i, ';'.join(topic)))



# graph-based representation of the document
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib notebook

# set the labels as list of candidates for each topic
labels = {i: ';'.join(topic) for i, topic in enumerate(extractor.topics)}

# set the weights of the edges
edge_weights = [extractor.graph[u][v]['weight'] for u,v in extractor.graph.edges()]

# set the weights of the nodes (topic weights are stored in _w attribute)
sizes = [10e3*extractor._w[i] for i, topic in enumerate(extractor.topics)]

# draw the graph
nx.draw_shell(extractor.graph, with_labels=True, labels=labels, width=edge_weights, node_size=sizes)


# representative candidate per topic
for candidate, weight in extractor.weights.items():
    print('{}: {}'.format(candidate, weight))



# Best candidates
keyphrases = extractor.get_n_best(n=7, stemming=False)

for i, (candidate, score) in enumerate(keyphrases):
    print("rank {}: {} ({})".format(i, candidate, score))
    graph.append(candidate)



graph

sentence 0:
 - words: Coronaviruses are a group of ...
 - stems: coronavirus are a group of ...
 - PoS: NOUN AUX DET NOUN ADP ...
sentence 1:
 - words: In humans and birds , ...
 - stems: in human and bird , ...
 - PoS: ADP NOUN CCONJ NOUN PUNCT ...
sentence 2:
 - words: Mild illnesses in humans include ...
 - stems: mild ill in human includ ...
 - PoS: ADJ NOUN ADP NOUN VERB ...
sentence 3:
 - words: In cows and pigs they ...
 - stems: in cow and pig they ...
 - PoS: ADP NOUN CCONJ NOUN PRON ...
sentence 4:
 - words: Coronaviruses constitute the subfamily Orthocoronavirinae ...
 - stems: coronavirus constitut the subfamili orthocoronavirina ...
 - PoS: NOUN VERB DET ADV PROPN ...
sentence 5:
 - words: They are enveloped viruses with ...
 - stems: they are envelop virus with ...
 - PoS: PRON AUX VERB NOUN ADP ...
sentence 6:
 - words: The genome size of coronaviruses ...
 - stems: the genom size of coronavirus ...
 - PoS: DET ADJ NOUN ADP NOUN ...
sentence 7:
 - words: They have charac

<IPython.core.display.Javascript object>

rna virus: 0.05338858918385088
rna genom: 0.021620375392983553
characterist appear: 0.011304379971948535
characterist club-shap spike: 0.011494658238801397
viral spike peplom: 0.0098179673756842
project: 0.021725011064902454
surfac: 0.01863658536475642
stellar corona: 0.02638561039543056
new speci: 0.014607364705881795
famili coronavirida: 0.020562075789531762
infect form: 0.01148566488589894
respiratori tract infect: 0.009910893880774985
mild: 0.01935368231081308
lethal: 0.019370525628173478
group: 0.018198586514656875
coronavirus: 0.034752011807004775
human: 0.02012859674711602
electron micrograph: 0.01927594002595458
alphacoronaviru: 0.011681015377054585
betacoronaviru: 0.012211610581141397
bird: 0.01876749602171329
borrow: 0.01099665374005794
case: 0.010389292285780446
common cold: 0.009412956464689443
coronaviru: 0.00869470105756571
covid-19: 0.011386619537663737
cow: 0.012217726131298397
crown: 0.00976293366590017
david tyrrel: 0.010635883940664548
deltacoronaviru: 0.011311018965

['rna viruses',
 'coronaviruses',
 'stellar corona',
 'project',
 'rna genome',
 'family coronaviridae',
 'humans']

# TextRank

In [8]:
extractor = pke.unsupervised.TextRank()
extractor.load_document(input=data , language='en')
extractor.candidate_selection()
extractor.candidate_weighting()
keyphrases = extractor.get_n_best(n=7, stemming=False)

for i, (candidate, score) in enumerate(keyphrases):
    print("rank {}: {} ({})".format(i, candidate, score))
    graph.append(candidate)


rank 0: common name coronavirus (0.05969013891914163)
rank 1: characteristic club-shaped spikes (0.05924626197089478)
rank 2: scientific name coronavirus (0.051447539586646)
rank 3: respiratory tract infections (0.04829079289811961)
rank 4: bulbous surface projections (0.04392673473975244)
rank 5: greek κορώνη korṓnē (0.04392590473975244)
rank 6: viral spike peplomers (0.0437897293716398)


# SingleRank Model 

In [9]:
extractor = pke.unsupervised.SingleRank()
extractor.load_document(input=data , language='en')
extractor.candidate_selection()
extractor.candidate_weighting()
keyphrases = extractor.get_n_best(n=7, stemming=False)

for i, (candidate, score) in enumerate(keyphrases):
    print("rank {}: {} ({})".format(i, candidate, score))
    graph.append(candidate)

rank 0: common name coronavirus (0.06249400863341042)
rank 1: rna viruses (0.05825894782251346)
rank 2: scientific name coronavirus (0.05674606898227093)
rank 3: genus name (0.04791251693837045)
rank 4: other viruses (0.04558668434451649)
rank 5: bulbous surface projections (0.04265387376606106)
rank 6: human coronaviruses (0.0419432231741188)


# PositionRank Model

In [10]:
extractor = pke.unsupervised.PositionRank()
extractor.load_document(input=data , language='en')
extractor.candidate_selection()
extractor.candidate_weighting()
keyphrases = extractor.get_n_best(n=7, stemming=False)

for i, (candidate, score) in enumerate(keyphrases):
    print("rank {}: {} ({})".format(i, candidate, score))
    graph.append(candidate)

rank 0: human coronaviruses (0.11487819155688117)
rank 1: rna viruses (0.08949725249385793)
rank 2: coronaviruses (0.08153823704403723)
rank 3: other viruses (0.06027172984859662)
rank 4: viruses (0.055168171454203455)
rank 5: rna genome (0.05222723006185316)
rank 6: common name coronavirus (0.04600820785334116)


In [11]:
graph

['rna viruses',
 'coronaviruses',
 'stellar corona',
 'project',
 'rna genome',
 'family coronaviridae',
 'humans',
 'common name coronavirus',
 'characteristic club-shaped spikes',
 'scientific name coronavirus',
 'respiratory tract infections',
 'bulbous surface projections',
 'greek κορώνη korṓnē',
 'viral spike peplomers',
 'common name coronavirus',
 'rna viruses',
 'scientific name coronavirus',
 'genus name',
 'other viruses',
 'bulbous surface projections',
 'human coronaviruses',
 'human coronaviruses',
 'rna viruses',
 'coronaviruses',
 'other viruses',
 'viruses',
 'rna genome',
 'common name coronavirus']

In [12]:
from collections import Counter
m=Counter(graph)
m.most_common(10)

[('rna viruses', 3),
 ('common name coronavirus', 3),
 ('coronaviruses', 2),
 ('rna genome', 2),
 ('scientific name coronavirus', 2),
 ('bulbous surface projections', 2),
 ('other viruses', 2),
 ('human coronaviruses', 2),
 ('stellar corona', 1),
 ('project', 1)]